# Using the P1 Agent


In [1]:
# First, we need to install autogen
!pip install autogen

In [2]:
import autogen
from planner import P1Agent, p1_reply
from termcolor import colored
import json
import time
from typing import Any, Callable, Coroutine, Dict, List, Literal, Optional, Tuple, Type, TypeVar, Union
import re
import json
import os

In [3]:
# Define the API
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    }
]

In [4]:
question = "If I invest $1 into QQQ every day before market close in 2022, how much money would I earn at the end of 2022?"

print(question)

If I invest $1 into QQQ every day before market close in 2022, how much money would I earn at the end of 2022?


## The Simple Question

> **Question**: If I invest \$1 into QQQ every day before market close in 2022, how much money would I earn at the end of 2022?

- ❌ *Both GPT-4 and O1 from ChatGPT.com failed to answer this question correctly.*
- ❓ *AutoGen Assistant Agent with GPT-4 and GPT-4o-mini can sometimes succeed with this question (try it yourself).*
- ✅ *In this tutorial, we will use GPT-4o-mini as the backbone for our O1Agent and successfully answer this question, outperforming the O1 model.*

**Note**: The code and answers are as of October 8, 2023, and GPT models may update over time.

### Challenges

- **Ambiguity in the Question**:
  - *Market Days*: The term "every day" is vague since stock markets are closed on weekends and holidays. In 2022, there were approximately 252 trading days, not 365.
  - *Definition of "Earn"*: Does "earn" refer to profit or the total value of holdings? Since the stock market declined in 2022, the final value is be negative.
- **Model Limitations**:
  - *Hallucinations*: Models like GPT-4o and O1 might confidently provide dollar values from memory without accessing real data. These numbers are very close but may not be exactly the same.
  - *Temporal Understanding*: Some models may misunderstand timeframes, especially regarding the "end of 2022."

### Ground Truth Answer:

The final profit should be around **-\$33** or **-13%**.

#### GPT-4o's Answer (in ChatGPT.com)
```
To determine how much money you would have earned by investing $1 daily in the QQQ (Invesco QQQ Trust) throughout 2022, we need to account for:

1. **Daily Closing Prices**: We need the QQQ closing prices for each trading day in 2022.
2. **Daily Investment**: We would invest $1 at each day's closing price to buy fractional shares.
3. **Total Shares and Value**: By the end of 2022, we can calculate the total value of all the shares accumulated by selling them at the closing price on the last trading day of 2022.

### Steps to Calculate:
1. Retrieve the daily closing prices for QQQ for each trading day in 2022.
2. Calculate the fractional shares bought each day by dividing $1 by the closing price for that day.
3. Sum the total shares accumulated by the end of the year.
4. Multiply the total accumulated shares by the closing price on the last trading day of 2022 to get the final value.

Do you have access to historical stock data, or would you like me to look up the 2022 QQQ daily prices to perform this calculation?
```

NOTE: if we prompt the question slightly differently, and ask GPT to write code or using search. Then, GPT-4o can solve this problem.

#### OpenAI O1's Answer (in ChatGPT.com)
Actually, o1 almost get the question correctly. However, because of lack of data access, its answer is inaccurate.
```
To determine how much money you would have at the end of 2022 if you invested \$1 into QQQ every trading day before market close, we'll follow these steps:

1. **Calculate the Number of Trading Days in 2022**:
   - There are typically around 252 trading days in a year. For 2022, accounting for weekends and market holidays, we'll use 252 as the number of trading days.
   - **Total Investment**: \$1/day × 252 days = **\$252**

2. **Gather Historical Closing Prices for QQQ in 2022**:
   - QQQ started 2022 at approximately **\$401.00** (closing price on December 31, 2021).
   - QQQ ended 2022 at approximately **\$266.28** (closing price on December 30, 2022).

3. **Calculate the Total Shares Purchased**:
   - Since you invest \$1 each day, the number of shares purchased each day is **\$1 divided by the closing price on that day**.
   - Due to the varying daily prices, you would buy more shares when the price is lower and fewer shares when the price is higher.

4. **Estimate the Average Price Paid per Share**:
   - As a simplification, we'll assume the average price of QQQ over 2022 is the average of the starting and ending prices:
     - Average Price ≈ (\$401.00 + \$266.28) / 2 ≈ **\$333.64**

5. **Calculate the Total Number of Shares Purchased**:
   - **Total Shares** ≈ Total Investment / Average Price ≈ \$252 / \$333.64 ≈ **0.7557 shares**

6. **Determine the Value of Your Investment at the End of 2022**:
   - **Final Value** = Total Shares × Ending Price ≈ 0.7557 shares × \$266.28 ≈ **\$201.32**

7. **Calculate Your Earnings or Loss**:
   - **Earnings/Loss** = Final Value - Total Investment ≈ \$201.32 - \$252 ≈ **-\$50.68**
   - This means you would have a **loss** of approximately **\$50.68**.

**Conclusion**:

By investing \$1 into QQQ every trading day in 2022, you would have invested a total of **\$252**. Due to the decline in QQQ's price over the year, your investment would be worth approximately **\$201.32** at the end of 2022, resulting in a loss of about **\$50.68**.

**Answer**: You would have about $201 at the end—a loss of approximately $51 on your $252 investment.
```

### Running the O1Agent

Below, we demonstrate how to run the O1Agent using GPT-4o-mini. Despite a lengthy trajectory and token usage, the cost remains relatively low due to the model's efficiency.

In [5]:
p1_agent = P1Agent(name="p1_agent", llm_config={"config_list": config_list}, verbose=True)
user_proxy = autogen.UserProxyAgent(name="user_proxy", human_input_mode="NEVER",
                                    code_execution_config={"use_docker":False},
                                   max_consecutive_auto_reply=10)

ans = user_proxy.initiate_chat(p1_agent, message=question, summary_method="reflection_with_llm")

user_proxy (to o1_agent):

If I invest $1 into QQQ every day before market close in 2022, how much money would I earn at the end of 2022?

--------------------------------------------------------------------------------
generate_response called from <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x00000298EF6D7610>
>>> Step 1: The chosen action is to gather historical daily closing prices of QQQ for 2022 to simulate the investment accurately. This is the most effective next step because without accurate pricing data, the simulation would not yield a correct or meaningful result.*

**RESULTS:**
*Once the historical daily closing prices are gathered, the simulation can be conducted to accurately determine the earnings, providing a clear answer to the user's question about their investment outcome.
>>> Step 2: The chosen action is to **Run the investment simulation using the gathered historical QQQ prices for 2022**. This is the most effective next step because it directly a

In [6]:
print(ans.summary)

Investing $1 into QQQ every day in 2022 resulted in a total investment of $251.00, with a final value of $217.52 at the end of the year, leading to a loss of $33.48. This outcome reflects the negative market performance of QQQ throughout 2022.


In [7]:
from autogen.agentchat.utils import gather_usage_summary
cost = gather_usage_summary([p1_agent, p1_agent.planner])
cost = sum([part["total_cost"] for part in cost.values()])

print(f"It takes ${cost:.5f} to get this answer.")

It takes $0.01549 to get this answer.


In [8]:
# equivalent call
p1_reply("How many 'r' in the word strawberry", config_list, verbose=True)

user_proxy (to o1_agent):

How many 'r' in the word strawberry

--------------------------------------------------------------------------------
generate_response called from <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x000002989670EB00>
>>> Step 1: The chosen action is to **count the occurrences of the letter 'r' in the word "strawberry."** This action is the most effective next step because it directly and efficiently addresses the user's question without unnecessary complexities.

**RESULTS:**
The word "strawberry" contains **three** occurrences of the letter 'r.'
>>> Step 2: I will choose to *Answer any follow-up questions that the user might have about word formation or letter frequencies* as the next step. This option has the highest score and allows for an interactive experience, catering to potential curiosity beyond the initial question.

**RESULTS:**
By inviting follow-up questions, I can assist the user further, ensuring clarity and promoting a deeper under

'The word "strawberry" has three occurrences of the letter \'r.\''